In [1]:
import h5py
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.signal import butter, filtfilt,detrend
import numpy as np
from numpy.polynomial import legendre as leg
from numpy.linalg import lstsq
import datetime
from datetime import timedelta
import pandas as pd
import geopy.distance
from shapely.geometry import LineString
import obspy
from obspy import UTCDateTime
from obspy.core import Trace
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame
from obspy.signal.trigger import classic_sta_lta,recursive_sta_lta
from obspy.signal.trigger import plot_trigger
import scipy
from scipy.fft import fft, ifft

In [2]:
# # Get a list of events from USGS and load events (specify search radius)
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime(2023, 1, 17, 0, 0),
                maxradiuskm=100, 
                latitude=47.60, 
                longitude=-122.33)



#Get list of USGS events
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['magnitude'],ascending=False)

#Create a list with the event id numbers for data download
event_df = event_df[(event_df['depth'] > 2.0) & (event_df['magnitude'] > 2.0)]
#culled = len(events)-len(event_df)
earthquake_list = event_df['id'].values.tolist()
#print("Found %s events" % len(events),"and culled %s"%culled, "events due to depth and Mw constraints")
event_df

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
474,uw61869652,2022-09-08 08:08:25.830,"7 km WNW of Lofall, Washington",47.842000,-122.751667,23.33,3.71,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,592
116,uw61835766,2022-05-01 11:15:56.460,"2 km E of Mount Vernon, Washington",48.423500,-122.305500,13.54,3.56,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,424
167,uw61832097,2022-05-17 08:07:34.400,"6 km ENE of Oso, Washington",48.299667,-121.852000,3.53,3.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,280
383,uw61860022,2022-08-04 13:22:39.400,"6 km SSE of Black Diamond, Washington",47.258667,-121.971833,21.54,3.06,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,250
290,uw61846212,2022-06-25 20:18:40.160,"5 km ENE of Oso, Washington",48.294167,-121.863500,6.29,2.95,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,154
249,uw61850651,2022-06-11 23:57:36.540,"34 km SSW of River Road, Washington",47.798500,-123.342000,41.05,2.81,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,122
747,uw61898306,2022-12-13 10:40:09.610,"4 km E of Lake Marcel-Stillwater, Washington",47.687333,-121.859000,25.66,2.68,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,115
283,uw61854526,2022-06-24 07:12:13.420,"4 km WNW of Quilcene, Washington",47.834000,-122.933167,52.20,2.64,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,108
401,uw61862587,2022-08-12 10:02:25.060,"24 km SSW of River Road, Washington",47.862167,-123.257167,43.45,2.56,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,101
305,uw61847792,2022-06-30 03:33:04.140,"3 km W of Hansville, Washington",47.922833,-122.606000,19.72,2.46,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,93


In [4]:
from dasquakes import *

data, times, attrs, x_max, this_id, data_filt, t0 = das_downloader(event_df,'uw61862587',cab='whidbey')

fiber_location, fiber_calibration, flat_x, flat_y = fiber_channel_locator(data, attrs,'fiberroute_whidbey.csv','calibration_whidbey.csv')

/data/data5/Converted/whidbey_2022-08-12_10-02*.h5
         Lat        Long  Elev  Description   Fiber Dist  Opt Dist  \
0  48.086666 -122.568055     0          NaN     0.000000      0.00   
1  48.101989 -122.575139     0          NaN  1783.640526   3353.20   
2  48.113849 -122.561772     0          NaN  3435.903037       NaN   
3  48.128587 -122.561947     0          NaN  5074.712547   6820.22   
4  48.135833 -122.511389     0          NaN  8923.254259  10975.00   

   Opt Dist Interp  Chan Num Interp  
0             0.00         0.000000  
1          3353.20       532.253968  
2          5086.71       807.414286  
3          6820.22      1082.574603  
4         10975.00      1742.063492  
         Lat        Long  Elev  Description   Fiber Dist  Opt Dist  \
0  48.086666 -122.568055     0          NaN     0.000000      0.00   
1  48.101989 -122.575139     0          NaN  1783.640526   3353.20   
2  48.113849 -122.561772     0          NaN  3435.903037       NaN   
3  48.128587 -122.56